In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import cross_val_score, KFold, ShuffleSplit,cross_val_predict
from sklearn import metrics

def from_features_to_classif_scores(features,labels,ftest,ltest):
    #stack them for learning
    features = features.reshape((len(features),-1))
    ftest = ftest.reshape((len(ftest),-1))
    # apply pipeline
    n = len(features)
    
    gs = GridSearchCV(SVC(kernel='rbf'), param_grad={'gamma',[0.01,0.05,0.1,0.11,0.15],'C',[0.8,0.9,1,1.1,1.2]})
    pipeline = make_pipeline(Normalizer(),StandardScaler(),gs)
    
    # pipeline = make_pipeline(Normalizer(),StandardScaler(),SVC(C=1.0, kernel='rbf', verbose=False))
    cv = ShuffleSplit(n,n_iter=3,test_size=1, train_size=1)
    
    scores = cross_val_score(pipeline,features,labels,cv=10)
    print("Accuracy: %0.2f (+/- %0.2f)" % ((1.0-scores.mean())*100, scores.std() * 2*100))

    return pipeline,scores

In [1]:
n = 60000
from load_mnist_db import load_scattering_mnist

import time as time
import numpy as np
from keras.datasets import mnist
num_images = n
px=32

Xtrain,ytrain,Xtest,ytest = load_scattering_mnist(num_images = num_images, px=px,J=3,L=6,m=1)
Xtrain_1d = Xtrain.reshape((len(Xtrain),-1))
num_samples,num_features = Xtrain_1d.shape



Loading images (in BW!!):
X_train shape: (60000, 32, 32)
60000 train samples
10000 test samples
60000  images loaded in  3.2200939655303955  secs
shape data: (60000, 32, 32)
Create filters:
Compute  60000  scatterings:
0 / 60000


/Users/ferradans/Documents/Research/ScatteringPython/scattering/scattering/scattering.py:46: ComplexWarning: Casting complex values to real discards the imaginary part
  out[:] = np.fft.rfft(img, axis=-1) * phi[0, :N_nolp_r].reshape(1, N_nolp_r)


500 / 60000
1000 / 60000
1500 / 60000
2000 / 60000
2500 / 60000
3000 / 60000
3500 / 60000
4000 / 60000
4500 / 60000
5000 / 60000
5500 / 60000
6000 / 60000
6500 / 60000
7000 / 60000
7500 / 60000
8000 / 60000
8500 / 60000
9000 / 60000
9500 / 60000
10000 / 60000
10500 / 60000
11000 / 60000
11500 / 60000
12000 / 60000
12500 / 60000
13000 / 60000
13500 / 60000
14000 / 60000
14500 / 60000
15000 / 60000
15500 / 60000
16000 / 60000
16500 / 60000
17000 / 60000
17500 / 60000
18000 / 60000
18500 / 60000
19000 / 60000
19500 / 60000
20000 / 60000
20500 / 60000
21000 / 60000
21500 / 60000
22000 / 60000
22500 / 60000
23000 / 60000
23500 / 60000
24000 / 60000
24500 / 60000
25000 / 60000
25500 / 60000
26000 / 60000
26500 / 60000
27000 / 60000
27500 / 60000
28000 / 60000
28500 / 60000
29000 / 60000
29500 / 60000
30000 / 60000
30500 / 60000
31000 / 60000
31500 / 60000
32000 / 60000
32500 / 60000
33000 / 60000
33500 / 60000
34000 / 60000
34500 / 60000
35000 / 60000
35500 / 60000
36000 / 60000
36500 / 6000

In [2]:
Xtrain_1d = Xtrain.reshape((len(Xtrain),-1))
Xtest_1d = Xtest.reshape((len(Xtest),-1))
Xtest_1d.shape

(10000, 1216)

In [ ]:
a = np.zeros((4,8,8))
b = np.ones((4,5,5))
a[:,2:7,2:7]=b
a[1,:,:]

In [ ]:
X_train, y_train, X_test, y_test = load_images_mnist(px=px)
print('Now testing set:')
wavelet_filters, littlewood = filter_bank_morlet2d(px, J=3, L=6)

scatterings_test =[]
step = 500
t_scats = time.time()
for i in np.arange(0, X_test.shape[0], step):
    print(i, '/', min(num_images, X_test.shape[0]))
    S,u = scattering(X_test[i:i + step , :, :], wavelet_filters)
    scatterings_test.append(S)

scatterings_test = np.concatenate(scatterings_test, axis=0)
print(scatterings_test.shape[0], ' scat. features computed in ', time.time() - t_scats , ' secs.')


In [ ]:
import os, sys
import scipy.misc
import scipy.io
print('saving data in ./mnist_scat_testing.mat')
scipy.io.savemat('./mnist_scat_testing.mat', mdict={'ytest': y_test, 'xtest': scatterings_test})
print('saving done!')


In [ ]:
import h5py
import numpy as np

f = h5py.File('mnist_scat_complet.mat')
Xtrain = np.array(f['xtrain']).transpose(3,2,1,0)
Xtest = np.array(f['xtest']).transpose(3,2,1,0)
ytrain = f['ytrain']
ytest = f['ytest']


In [ ]:
Xtrain_1d = Xtrain.reshape((len(Xtrain),-1))
Xtrain.shape
Xtest_1d = Xtest.reshape((len(Xtest),-1))
Xtest_1d.shape

In [ ]:
Xtrain_1d.shape

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, Normalizer

n = 300
parameters = {"C":[1]}

pip_linear = make_pipeline(StandardScaler(),SVC(kernel='linear',C=1))

pip_linear.fit(Xtrain_1d[0:n,:],ytrain[0:n])
out=pip_linear.predict(Xtest_1d)
score = accuracy_score(ytest, out)
score

In [ ]:
Xtrain_1d[0:n,:].shape
ytrain[0:n].shape
gs_gaussian

In [3]:
def gethomogeneus_datast(X,y,d,n):
    num_per_class = np.int(n/d)
    ytrain=np.reshape(y,(y.shape[0],))

    X_out = []#np.zeros(n,X.shape[1],X.shape[2],X.shape[3])
    y_out = []
    for i_d in np.arange(d):
        indx = np.where(ytrain.ravel()==i_d)

        X_out.append(X[indx[0][0:num_per_class],:])
        y_out.append(ytrain[indx[0][0:num_per_class],])

    
    X_out = np.concatenate(X_out,axis=0)
    y_out = np.concatenate(y_out,axis=0)
    #print(X_out.shape)
    #print(y_out)
    return X_out,y_out


In [ ]:
print(bestgamma)
print(bestC)

In [5]:
np.arange(0.8,4,0.5).shape

(7,)

In [ ]:
##### Gaussian kernel grid search
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer


gammavec = 10**np.arange(-3,-0,0.5)#10**np.arange(-2.25,-2.15,0.01) got 0.00616595 #np.array(10**np.arange(-3,-0.5,0.1)) got 0.0063095734448
gammavec.shape = (6)
Cvec =  np.arange(0.8,4,0.5) #np.arange(3.15,3.55,0.10) got 3.25 #np.arange(0.5,4,0.25) got 3.25
Cvec.shape=(7)
parameters = {"C":Cvec,
              "gamma":gammavec}

gs_gaussian = GridSearchCV(SVC(kernel='rbf'),parameters)
pip_gaussian = make_pipeline(MinMaxScaler((-1,1)),Normalizer(),gs_gaussian)
n = 4000
Xa,ya=gethomogeneus_datast(Xtrain_1d,ytrain,10,n)

np.histogram(ya)

pip_gaussian.fit(Xa,ya.ravel())

bestC = gs_gaussian.best_params_['C']
bestgamma = gs_gaussian.best_params_['gamma']
print(bestgamma)
print(bestC)


In [ ]:
print(np.log10(bestgamma))
gammavec
print(10**np.arange(-2.25,-2.15,0.01))
#print(gammavec)
print(bestgamma)

In [ ]:
print(bestC)
Cvec
np.arange(3.15,3.55,0.10)

In [ ]:
########### Gaussian kernel methods

from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler, Normalizer

num_items,num_features = Xtrain_1d.shape

ns = [300, 1000, 2000, 5000, 10000,20000]
score_gaussian = np.zeros((len(ns),1))
for i,n in enumerate(ns):
    #  gs_gaussian = GridSearchCV(SVC(kernel='rbf'),param_grid=parameters)
    gs_gaussian = SVC(kernel='rbf',C=bestC,gamma=bestgamma)
    pip_gaussian = make_pipeline(MinMaxScaler((-1,1)),Normalizer(),gs_gaussian)

#    gs_gaussian = SVC(kernel='rbf',gamma=0.0018)
#    pip_gaussian = make_pipeline(StandardScaler(),Normalizer(),gs_gaussian)

    Xa,ya=gethomogeneus_datast(Xtrain_1d,ytrain,10,n) 
    pip_gaussian.fit(Xa,ya)
    out=pip_gaussian.predict(Xtest_1d)
    score_gaussian[i] = 1.0-accuracy_score(ytest, out)
    print(score_gaussian[i])
    
score_gaussian


In [ ]:
print(bestC)
print(bestgamma)

In [ ]:
np.arange(-3,-0.5,0.1).shape
n = 10000
Xa,ya=gethomogeneus_datast(Xtrain_1d,ytrain,10,n) 


In [ ]:
1/2032

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler, Normalizer

gammavec = np.array(10**np.arange(0.46,0.52,0.01)) #np.array(10**np.arange(-0.1,1,0.3))  best 0.5 #np.array(10**np.arange(-3,-0.5,0.1)) got 0.0063095734448
gammavec.shape = (6)
Cvec = np.arange(3.8,3.95,0.05)
#Cvec.append(1) #np.arange(3.15,3.55,0.10) #np.arange(0.5,4,0.25) got 3.25
Cvec.shape=(4)
parameters = {"C":Cvec,
              "gamma":gammavec}

gs_gaussian = GridSearchCV(SVC(kernel='rbf'),parameters)
pip_gaussian = make_pipeline(MinMaxScaler((-1,1)),Normalizer(),gs_gaussian)
pip_gaussian.fit(Xa,ya)

bestC = gs_gaussian.best_params_['C']
bestgamma = gs_gaussian.best_params_['gamma']
print(bestC)
print(bestgamma)
gs_gaussian = SVC(kernel='rbf',C=bestC,gamma=bestgamma)
pip_gaussian = make_pipeline(MinMaxScaler((-1,1)),Normalizer(),gs_gaussian)

pip_gaussian.fit(Xa,ya)
out=pip_gaussian.predict(Xtest_1d)
score = 1.0-accuracy_score(ytest, out)
print(score) # C=3, gamma = 3.16227766, score=0.0139
#C=3.9, gamma = 2.88403150313 score=0.0137

In [ ]:
#score_gaussian[i] = 1.0-accuracy_score(ytest, out)
#print(score_gaussian)
print(gammavec)
Xa.shape
gs_gaussian.best_params_

In [ ]:
### linear

from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

num_items,num_features = Xtrain_1d.shape

ns = [300, 1000, 2000, 5000, 10000]
score_gaussian = np.zeros((5,1))
for i,n in enumerate(ns):
    #  gs_gaussian = GridSearchCV(SVC(kernel='rbf'),param_grid=parameters)
    gs_gaussian = SVC(kernel='linear',C=1)
    pip_gaussian = make_pipeline(MinMaxScaler((-1,1)),gs_gaussian)

#    gs_gaussian = SVC(kernel='rbf',gamma=0.0018)
#    pip_gaussian = make_pipeline(StandardScaler(),Normalizer(),gs_gaussian)

    Xa,ya=gethomogeneus_datast(Xtrain_1d,ytrain,10,n)    
    pip_gaussian.fit(Xa,ya)
    out=pip_gaussian.predict(Xtest_1d)
    score_gaussian[i] = 1.0-accuracy_score(ytest, out)
    print(score_gaussian[i])
    
score_gaussian


In [ ]:
Xtest_1d = Xtest.reshape((len(Xtest),-1))
Xe = normalize(Xtest_1d)
Xe = scale(Xe)

predictedY = gs_linear.predict(Xe)
score = accuracy_score(ytest, predictedY)
score

In [ ]:

parameters = {"C":[1,1.1,1.11,1.05] ,
              "gamma":[0.1/num_features,1/num_features,10./num_features]}
gs_gaussian = GridSearchCV(SVC(kernel='rbf'),param_grid=parameters)

X = normalize(Xtrain_1d[0:n,:])
X = scale(X)

gs_gaussian.fit(X,ytrain[0:n])
gs_gaussian.best_params_

Xtest_1d = Xtest.reshape((len(Xtest),-1))
Xe = normalize(Xtest_1d)
Xe = scale(Xe)

predictedY = gs_gaussian.predict(Xe)
score = accuracy_score(ytest, predictedY)
score


In [ ]:
from sklearn.metrics import accuracy_score
print('mine:',np.sum(ytest==predictedY)/len(Xtest))
print('two:',accuracy_score(ytest, predictedY))

In [ ]:

#sampling the whole dataset 
n = 300
train_size=n/60000
test_size=0.5*n/60000

pipeline = make_pipeline(Normalizer(),StandardScaler(),gs)

# pipeline = make_pipeline(Normalizer(),StandardScaler(),SVC(C=1.0, kernel='rbf', verbose=False))
cv = ShuffleSplit(n,n_iter=3,test_size=test_size, train_size=train_size)

scores = cross_val_score(pipeline,Xtrain_1d[0:n,:],ytrain[0:n],cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % ((1.0-scores.mean())*100, scores.std() * 2*100))


In [ ]:
gs.best_params_
pipeline.

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import cross_val_score, KFold, ShuffleSplit,cross_val_predict
from sklearn import metrics

def from_features_to_classif_scores(features,labels,ftest,ltest):
    #stack them for learning
    features = features.reshape((len(features),-1))
    ftest = ftest.reshape((len(ftest),-1))
    # apply pipeline
    n = len(features)
    
    pipeline = make_pipeline(Normalizer(),StandardScaler(),SVC(C=1.0, kernel='rbf', verbose=False))
    cv = ShuffleSplit(n,n_iter=3,test_size=0.1, train_size=0.9)
    
    scores = cross_val_score(pipeline,features,labels,cv=10)
    print("Accuracy: %0.2f (+/- %0.2f)" % ((1.0-scores.mean())*100, scores.std() * 2*100))
    print('min score:',(1.0-scores.max())*100)
    return pipeline,scores

In [ ]:
from load_mnist_db import load_scattering_mnist
import time as time
import numpy as np
from keras.datasets import mnist

num_images = 60000
px=32

Xtrain,ytrain,Xtest,ytest = load_scattering_mnist(num_images = num_images, px=px,J=3,L=8)

num_points = [330, 1100, 2200, 5500, 11000, 22000, 60000]
for n in num_points:
    print('n=',n)
    pipeline, test_err = from_features_to_classif_scores(Xtrain[0:n,:,:,:],ytrain[0:n],Xtest[0:n,:,:,:],ytest[0:n])


In [ ]:
test_err

In [ ]:
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.svm import SVC

X_train = Xtrain
y_train = ytrain
X_train = features.reshape((len(X_train),-1))
X_test = Xtest.reshape((len(Xtest),-1))

# apply pipeline
n = len(X_train)

pipeline = make_pipeline(Normalizer(),StandardScaler(),SVC(C=1.0, kernel='rbf', verbose=True))
#cv = ShuffleSplit(n,n_iter=3,test_size=1, train_size=1)

normalizer = Normalizer()
scaler = StandardScaler().fit(X_train)
X_train_normalized = normalizer.transform(X_train)
X_train_transformed = scaler.transform(X_train_normalized)

clf = SVC(kernel='rbf', C=1, verbose=True).fit(X_train_transformed, y_train)

X_test_normalized = normalizer.transform(X_test)
X_test_transformed = scaler.transform(X_test_normalized)

clf.score(Xtest, ytest)   


print('score:',scores)